In [28]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')
train.shape

(156060, 4)

In [29]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def phrase_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    phrase_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    phrase_text = re.sub("[^a-zA-Z]"," ", phrase_text)
    #
    # 3. Convert words to lower case and split them
    words = phrase_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return words

In [30]:
trainPhrases = []
for phrase in train.Phrase:
    trainPhrases.append(phrase_to_wordlist(phrase))

print len(trainPhrases)

testPhrases = []
for phrase in test.Phrase:
    testPhrases.append(phrase_to_wordlist(phrase))
    
print len(testPhrases)

156060
66292


In [31]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(trainPhrases, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...


In [51]:
model.most_similar("no")

[(u'doubt', 0.6463204026222229),
 (u'reason', 0.6391289830207825),
 (u'here', 0.6342751383781433),
 (u'problem', 0.6216509342193604),
 (u'plenty', 0.6104859709739685),
 (u'rewards', 0.4606524407863617),
 (u'harris', 0.45735523104667664),
 (u'maybe', 0.4547345042228699),
 (u'nothing', 0.45289337635040283),
 (u'amount', 0.45206189155578613)]

In [37]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs


In [38]:
trainVecs = getAvgFeatureVecs(trainPhrases, model, 300)
testVecs = getAvgFeatureVecs(testPhrases, model, 300)

Review 0 of 156060
Review 1000 of 156060
Review 2000 of 156060
Review 3000 of 156060
Review 4000 of 156060
Review 5000 of 156060
Review 6000 of 156060
Review 7000 of 156060
Review 8000 of 156060
Review 9000 of 156060
Review 10000 of 156060
Review 11000 of 156060
Review 12000 of 156060
Review 13000 of 156060
Review 14000 of 156060
Review 15000 of 156060
Review 16000 of 156060
Review 17000 of 156060
Review 18000 of 156060
Review 19000 of 156060
Review 20000 of 156060
Review 21000 of 156060
Review 22000 of 156060
Review 23000 of 156060
Review 24000 of 156060
Review 25000 of 156060
Review 26000 of 156060
Review 27000 of 156060
Review 28000 of 156060
Review 29000 of 156060
Review 30000 of 156060
Review 31000 of 156060
Review 32000 of 156060
Review 33000 of 156060
Review 34000 of 156060
Review 35000 of 156060
Review 36000 of 156060
Review 37000 of 156060
Review 38000 of 156060
Review 39000 of 156060
Review 40000 of 156060
Review 41000 of 156060
Review 42000 of 156060
Review 43000 of 156060
R

In [39]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainVecs, train.Sentiment)

# Test & extract results 
result = forest.predict( testVecs )

# Write the test results 
output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': result
})
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [40]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  16.08842206 seconds.


In [41]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [42]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'minds', u'mentally', u'souls', u'perspective', u'knowledge']

Cluster 1
[u'terrific', u'escapism', u'attractive']

Cluster 2
[u'skip', u'visit', u'rent', u'theatre', u'create', u'buy']

Cluster 3
[u'dry', u'score', u'animation', u'humor', u'creative', u'verve', u'visual', u'imagination', u'style', u'purpose', u'wit', u'excitement']

Cluster 4
[u'edgy', u'ill', u'impressive', u'violent', u'oddly', u'sensual', u'throughout', u'energetic', u'irritating', u'unpredictable', u'hilarious', u'intermittently', u'deceptively', u'inventive']

Cluster 5
[u'ahead']

Cluster 6
[u'unconventional', u'watchable', u'damn', u'decent', u'offensive']

Cluster 7
[u'extreme', u'packed', u'pic']

Cluster 8
[u'artificial', u'unexpectedly', u'engrossing', u'unusual', u'deeply', u'fellow', u'intense', u'portrait', u'undeniably', u'poignant']

Cluster 9
[u'traditions', u'sketchy', u'severe', u'terrorism']


In [43]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [44]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train.Phrase.size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for phrase in trainPhrases:
    train_centroids[counter] = create_bag_of_centroids( phrase, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test.Phrase.size, num_clusters), \
    dtype="float32" )

counter = 0
for phrase in testPhrases:
    test_centroids[counter] = create_bag_of_centroids( phrase, \
        word_centroid_map )
    counter += 1

In [45]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids, train.Sentiment)
result = forest.predict(test_centroids)
 
# Write the test results 
output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': result
})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
